In [ ]:
pip install wfdb neurokit2 biosppy dtcwt heartpy pandas-profiling

In [ ]:
# !wget -r -N -c -np https://physionet.org/files/bidmc/1.0.0/

In [ ]:
import biosppy
import numpy as np
import wfdb
import neurokit2 as nk
from collections import Counter
import dtcwt
from scipy.fft import fft
import matplotlib.pyplot as plt


In [ ]:
records = [
'bidmc01',
'bidmc02',
'bidmc03',
'bidmc04',
'bidmc05',
'bidmc06',
'bidmc07',
'bidmc08',
'bidmc09',
'bidmc10',
'bidmc11',
'bidmc12',
'bidmc13',
'bidmc14',
'bidmc15',
'bidmc16',
'bidmc17',
'bidmc18',
'bidmc19',
'bidmc20',
'bidmc21',
'bidmc22',
'bidmc23',
'bidmc24',
'bidmc25',
'bidmc26',
'bidmc27',
'bidmc28',
'bidmc29',
'bidmc30',
'bidmc31',
'bidmc32',
'bidmc33',
'bidmc34',
'bidmc35',
'bidmc36',
'bidmc37',
'bidmc38',
'bidmc39',
'bidmc40',
'bidmc41',
'bidmc42',
'bidmc43',
'bidmc44',
'bidmc45',
'bidmc46',
'bidmc47',
'bidmc48',
'bidmc49',
'bidmc50',
'bidmc51',
'bidmc52',
'bidmc53'
]


In [ ]:
def classifyingforclass(data,datahr,datares,datasp):
    hr = sum(datahr) / len(datahr)
    bpm = sum(data) // len(data)
    res = sum(datares) / len(datares)
    sp = sum(datasp) / len(datasp)
    if bpm in range(0,81):
        return [hr,bpm,res,sp,0]
    elif bpm in range(80,90):
        return [hr,bpm,res,sp,1]
    elif bpm in range(90,100):
        return [hr,bpm,res,sp,2]
    elif bpm in range(100,200):
        return [hr,bpm,res,sp,3]
    else:
        return [hr,bpm,res,sp]

In [ ]:
datasetall = []
track = [i for i in range(0,60100,250)]
track1 = [i for i in range(0,481,2)]
print(track)
print(track1)
print(len(track))
print(len(track1))

[0, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000, 4250, 4500, 4750, 5000, 5250, 5500, 5750, 6000, 6250, 6500, 6750, 7000, 7250, 7500, 7750, 8000, 8250, 8500, 8750, 9000, 9250, 9500, 9750, 10000, 10250, 10500, 10750, 11000, 11250, 11500, 11750, 12000, 12250, 12500, 12750, 13000, 13250, 13500, 13750, 14000, 14250, 14500, 14750, 15000, 15250, 15500, 15750, 16000, 16250, 16500, 16750, 17000, 17250, 17500, 17750, 18000, 18250, 18500, 18750, 19000, 19250, 19500, 19750, 20000, 20250, 20500, 20750, 21000, 21250, 21500, 21750, 22000, 22250, 22500, 22750, 23000, 23250, 23500, 23750, 24000, 24250, 24500, 24750, 25000, 25250, 25500, 25750, 26000, 26250, 26500, 26750, 27000, 27250, 27500, 27750, 28000, 28250, 28500, 28750, 29000, 29250, 29500, 29750, 30000, 30250, 30500, 30750, 31000, 31250, 31500, 31750, 32000, 32250, 32500, 32750, 33000, 33250, 33500, 33750, 34000, 34250, 34500, 34750, 35000, 35250, 35500, 35750, 36000, 36250, 36500, 36750, 37000, 37

In [ ]:
# Normal = 0
# Prehypertension = 1
# Stage 1 Hypertension = 2
# Stage 2 Hypertension = 3



path = "/content/physionet.org/files/bidmc/1.0.0/"

allsignal = []
allfeature = []

for i in records:

    signals, fields = wfdb.rdsamp(path+i)
    data0 = signals[:,[1]].ravel()
    subsignal=[]

    signalsn, fieldsn = wfdb.rdsamp(path+i+'n')
    data = signalsn[:,[1]].ravel()
    datahr = signalsn[:,[0]].ravel()
    datares = signalsn[:,[2]].ravel()
    datasp = signalsn[:,[3]].ravel()

    for j in range(0,len(track)-1):
        prev = track[j]
        next = track[j+1]
        bidmc01_sub = data0[prev:next]
        allsignal.append(bidmc01_sub)

    for k in range(0,len(track1)-1):
        data1 = data[~np.isnan(data)][k:k+1]
        data1hr = datahr[~np.isnan(datahr)][k:k+1]
        data1res = datares[~np.isnan(datares)][k:k+1]
        data1sp = datasp[~np.isnan(datasp)][k:k+1]
        bidmc01_sub1_class = classifyingforclass(data1,data1hr,data1res,data1sp)
        allfeature.append(bidmc01_sub1_class)



In [ ]:
print(len(allsignal))
print(len(allfeature))


12720
12720


In [ ]:
for i in range(0,len(allsignal)):
    print(i)
    fourFeatures = biosppy.signals.tools.signal_stats(signal=allsignal[i])

    # getting signal data
    vecs = allsignal[i]

    #4th scale and 5th scale data
    fourthscale = None
    fifthscale = None
    # 1D transform, 5 levels
    transform = dtcwt.Transform1d()
    vecs_t = transform.forward(vecs, nlevels=5)
    #assiging 4th scale and 5th scale data
    fourthscale = np.abs(vecs_t.highpasses[3])
    fifthscale = np.abs(vecs_t.highpasses[4])
    # Inverse
    vecs_recon = transform.inverse(vecs_t)
    # 1D FFT and taking log spectrum values
    #4th scale and 5th scale data after 1D FFT
    fourthFFT = None
    fifthFFT = None
    #assiging 4th scale and 5th scale data after 1D FFT
    fourthFFT = fft(vecs_t.highpasses[3]).ravel()
    fifthFFT = fft(vecs_t.highpasses[4]).ravel()
    #concating forth and fifth FFT data
    totalFFT = None
    totalFFT = np.concatenate((fourthFFT,fifthFFT),axis=0).ravel().tolist()
    totalFFT.extend([fourFeatures[-2],fourFeatures[-1]])
    totalFFT.extend(allfeature[i])

    #adding all features to dict and adding class
    Totalfeat = totalFFT


    datasetall.append(Totalfeat)






In [ ]:
data = np.asarray(datasetall)

In [ ]:
np.save('BIDMCPPGREP250.npy', data)

In [ ]:
data = np.load('BIDMCPPGREP250.npy')

In [ ]:
data

In [ ]:
import pandas as pd
BIDMCPPGREP_df = pd.DataFrame(data=data,dtype="float64")
BIDMCPPGREP_df = BIDMCPPGREP_df.dropna()
BIDMCPPGREP_df

In [ ]:
y=BIDMCPPGREP_df.iloc[:,-1]
y.unique()

array([2., 1., 0., 3.])

In [ ]:
import pandas as pd
df = pd.read_csv('/content/PPGBPHT.csv')
#df = df.drop(['Num.','Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],axis=1)
#df['Sex(M/F)'] = df['Sex(M/F)'].astype('category')
#cat_columns = df.select_dtypes(['category']).columns
#df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df

,Num.,subject_ID,Sex(M/F),Age(year),Height(cm),Weight(kg),Systolic Blood Pressure(mmHg),Diastolic Blood Pressure(mmHg),Heart Rate(b/m),BMI(kg/m^2),Hypertension
0,1,2,0,45,152,63,161,89,97,27.27,4
1,2,3,0,50,157,50,160,93,76,20.28,4
2,3,6,0,47,150,47,101,71,79,20.89,1
3,4,8,1,45,172,65,136,93,87,21.97,2
4,5,9,0,46,155,65,123,73,73,27.06,2
...,...,...,...,...,...,...,...,...,...,...,...
214,215,415,1,24,180,70,111,70,77,21.60,1
215,216,416,0,25,156,47,93,57,79,19.31,1
216,217,417,1,25,176,55,120,69,72,17.76,2
217,218,418,1,25,173,63,106,69,67,21.05,1


In [ ]:
from scipy import signal
import os

dir = '/content/drive/My Drive/ppg data set/Data File/Alldata'

totaldata = []
for filename in os.listdir(dir):

    if filename.endswith(".txt"):
        readdata = np.loadtxt(os.path.join(dir, filename))
        filtered = biosppy.signals.tools.filter_signal(signal=readdata, ftype='butter', band='bandpass', order=3,frequency=[0.5,4], sampling_rate=1000.0)[0]

        if len(readdata) == 2100:

                a = filename.split('_')[0]

                temp = df.loc[df['subject_ID'] == int(a)]
                features = temp.values.tolist()[0][1:]
                #classlabel = datadist[a]
                #uniclass = unicodelavel[classlabel]

                fourFeatures = biosppy.signals.tools.signal_stats(signal=filtered)

                # getting signal data
                vecs = filtered

                #4th scale and 5th scale data
                fourthscale = None
                fifthscale = None
                # 1D transform, 5 levels
                transform = dtcwt.Transform1d()
                vecs_t = transform.forward(vecs, nlevels=5)
                #assiging 4th scale and 5th scale data
                fourthscale = np.abs(vecs_t.highpasses[3])
                fifthscale = np.abs(vecs_t.highpasses[4])
                # Inverse
                vecs_recon = transform.inverse(vecs_t)
                # 1D FFT and taking log spectrum values
                #4th scale and 5th scale data after 1D FFT
                fourthFFT = None
                fifthFFT = None
                #assiging 4th scale and 5th scale data after 1D FFT
                fourthFFT = fft(vecs_t.highpasses[3]).ravel()
                fifthFFT = fft(vecs_t.highpasses[4]).ravel()
                #concating forth and fifth FFT data
                totalFFT = None
                totalFFT = np.concatenate((fourthFFT,fifthFFT),axis=0).ravel().tolist()
                totalFFT.extend([fourFeatures[-2],fourFeatures[-1]])
                totalFFT.extend(features)


                #adding all features to dict and adding class
                Totalfeat = totalFFT
                totaldata.append(Totalfeat)
                print(filename)



179_2.txt
176_3.txt
178_3.txt
180_2.txt
179_1.txt
178_1.txt
179_3.txt
183_2.txt
184_1.txt
182_1.txt
182_3.txt
184_2.txt
185_1.txt
184_3.txt
183_3.txt
182_2.txt
183_1.txt
190_3.txt
188_3.txt
188_1.txt
189_2.txt
186_2.txt
188_2.txt
186_1.txt
189_3.txt
186_3.txt
189_1.txt
190_1.txt
185_3.txt
190_2.txt
185_2.txt
193_2.txt
191_3.txt
193_3.txt
192_3.txt
191_1.txt
191_2.txt
193_1.txt
195_3.txt
195_2.txt
196_2.txt
192_2.txt
192_1.txt
196_1.txt
195_1.txt
198_1.txt
197_3.txt
198_3.txt
198_2.txt
199_3.txt
199_1.txt
199_2.txt
197_2.txt
196_3.txt
197_1.txt
201_1.txt
201_3.txt
201_2.txt
200_2.txt
200_3.txt
205_2.txt
206_1.txt
205_3.txt
203_2.txt
203_1.txt
203_3.txt
205_1.txt
200_1.txt
209_1.txt
210_1.txt
209_3.txt
206_2.txt
210_3.txt
207_3.txt
208_3.txt
207_2.txt
208_1.txt
210_2.txt
208_2.txt
206_3.txt
209_2.txt
207_1.txt
211_2.txt
213_3.txt
213_2.txt
212_3.txt
212_2.txt
212_1.txt
213_1.txt
214_3.txt
214_2.txt
211_3.txt
215_2.txt
214_1.txt
211_1.txt
215_1.txt
220_1.txt
218_2.txt
219_1.txt
216_1.txt


In [ ]:
np.save('"Published/FIGPPGBP.npy', totaldata)

In [ ]:
data = np.load('FIGPPGBP.npy')

In [ ]:
import numpy as np
import math
from numpy import genfromtxt
import neurokit2 as nk
import biosppy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 5]


def find_S_point(ecg, R_peaks):
	num_peak=len(R_peaks)
	S_point=[]
	for index in range(num_peak):
		i=R_peaks[index]
		cnt=i
		if cnt+1>=ecg.shape[0]:
			break
		while ecg[cnt]>ecg[cnt+1]+1:
			cnt+=1
			print(cnt)
			if cnt>=ecg.shape[0]:
				break
		S_point.append(cnt)
	#print('S_point',S_point)
	return S_point


def find_Q_point(ecg, R_peaks):
	num_peak=len(R_peaks)
	Q_point=[]
	for index in range(num_peak):
		i=R_peaks[index]
		cnt=i
		if cnt-1<0:
			break
		while ecg[cnt]>ecg[cnt-1]:
			cnt-=1
			if cnt<0:
				break
		Q_point.append(cnt)
	#print('Qpoint',Q_point)
	return Q_point

def find_R_peaks(ecg,fs):
	#R_peaks=detectors.pan_tompkins_detector(ecg)
	signal = ecg
	R_peaks =  biosppy.signals.ecg.christov_segmenter(signal=signal, sampling_rate=fs)
	#nk.ecg_peaks(ecg, sampling_rate=360).ravel()[0:1000]
	#print('R_peaks',R_peaks[0])
	return R_peaks[0].ravel()[0:1000]



def EKG_QRS_detect(ecg, fs, QS, plot=False):
	sig_lgth=len(ecg)


	# ecg_lgth_transform=lgth_transform(ecg_lgth_transform, int(fs/40))



	#peaks=find_peak(ecg, 128)

	R_peaks = find_R_peaks(ecg,fs)
	if QS:
		S_point=find_S_point(ecg, R_peaks)
		Q_point=find_Q_point(ecg, R_peaks)
	else:
		S_point=None
		Q_point=None
	if plot:
		#print(['R_peaks',R_peaks,'S_point',S_point,'Q_point',Q_point])
		plot = nk.events_plot([Q_point,R_peaks,S_point], ecg)
	return R_peaks,S_point,Q_point

In [ ]:
import pywt

In [ ]:
signals, fields = wfdb.rdsamp('100')
signal = signals.ravel()[0:2000]
filsig = biosppy.signals.tools.filter_signal(signal=signal, ftype='butter', band='bandpass', order=3, frequency=[4,22], sampling_rate=369.0)[0]
R_peak, S_point, Q_point = EKG_QRS_detect(filsig, 360, True, False)
S_point = [168, 756, 1339, 1906]
print(R_peak,S_point,Q_point)
diff = np.diff(filsig)
squared = diff*diff

[ 152  739 1324 1891] [168, 756, 1339, 1906] [133, 718, 1305, 1871]


In [ ]:
def MWA_cumulative(input_array, window_size):

    ret = np.cumsum(input_array, dtype=float)
    ret[window_size:] = ret[window_size:] - ret[:-window_size]

    for i in range(1,window_size):
        ret[i-1] = ret[i-1] / i
    ret[window_size - 1:]  = ret[window_size - 1:] / window_size

    return ret

N = int(0.12*360)
mwa = MWA_cumulative(squared, N)
mwa[:int(0.2*360)] = 0

def find_R_peaks(ecg,fs):
	#R_peaks=detectors.pan_tompkins_detector(ecg)
	signal = ecg
	R_peaks =  biosppy.signals.ecg.christov_segmenter(signal=signal, sampling_rate=fs)
	#nk.ecg_peaks(ecg, sampling_rate=360).ravel()[0:1000]
	#print('R_peaks',R_peaks[0])
	return R_peaks[0].ravel()[0:1000]

mR_peaks = find_R_peaks(mwa,360)
print(mR_peaks)

In [ ]:
%matplotlib inline

rbeat = [filsig[x] for x in R_peak] #Get the y-value of all peaks for plotting purposes
sbeat = [filsig[y] for y in S_point]
qbeat = [filsig[z] for z in Q_point]

mrbeat = [mwa[xx] for xx in mR_peaks]

fig, ax = plt.subplots(5)
fig.set_size_inches(13,20)

ax[0].plot(signal)
ax[0].set_xlim(0,2000)
ax[0].set_title('(a)')

ax[1].plot(filsig)
ax[1].set_xlim(0,2000)
ax[1].set_title('(b)')

ax[2].plot(squared)
ax[2].set_xlim(0,2000)
ax[2].set_title('(c)')


ax[3].plot(mwa)
ax[3].plot(mR_peaks, mrbeat,'ro',markersize=20,fillstyle='none')
ax[3].set_xlim(0,2000)
ax[3].set_title('(d)')

ax[4].plot(filsig, alpha=0.5, color='blue')
ax[4].scatter(R_peak, rbeat,color='blue' )
ax[4].scatter(S_point, sbeat,color='red' )
ax[4].scatter(Q_point, qbeat,color='green')
ax[4].set_title('(e)')
ax[4].set_xlim(0,2000)



fig.savefig('pantomkinsni.png', dpi=300)
plt.show()

In [ ]:
coeff = pywt.wavedec(signals.ravel(), 'db2',level=1)
cA1,cD1 = coeff

In [ ]:
coeffs = pywt.wavedec(signals.ravel(), 'db2',level=2)
cA2,cD2,cD1 = coeffs

In [ ]:
coeffss = pywt.wavedec(signals.ravel(), 'db2',level=3)
cA3,cD3,cD2,cD1 = coeffss

In [ ]:
plt.savefig('D1new.png', dpi=300)

In [ ]:
plt.plot(cA1)
plt.xlim(0,1000)
plt.ylim(-1.8,1.8)
plt.savefig('A1new.png', dpi=300)
plt.show()
plt.plot(cD1)
plt.xlim(0,1000)

plt.savefig('D1new.png', dpi=300)
plt.show()

plt.plot(cA2)
plt.xlim(0,1000)
plt.ylim(-1.8,1.8)
plt.savefig('A2new.png', dpi=300)
plt.show()
plt.plot(cD2)
plt.xlim(0,1000)

plt.savefig('D2new.png', dpi=300)
plt.show()

plt.plot(cA3)
plt.xlim(0,1000)
plt.ylim(-2.3,2.3)
plt.savefig('A3new.png', dpi=300)
plt.show()
plt.plot(cD3)
plt.xlim(0,1000)

plt.savefig('D3new.png', dpi=300)
plt.show()